In [1]:
import pandas as pd
import numpy as np
import glob
import os

def cargar_con_origen(file):
    df = pd.read_csv(file)
    df['source_file'] = os.path.basename(file)  
    return df

file_2022 = sorted(glob.glob("./data/2022/limpio/*.csv"))
file_2023 = sorted(glob.glob("./data/2023/limpio/*.csv"))
file_2024 = sorted(glob.glob("./data/2024/limpio/*.csv"))

all_data = pd.concat([cargar_con_origen(file) for file in file_2022 + file_2023 + file_2024], ignore_index=True)

In [2]:
all_data

,juego,9:00 ciclos,9:00 asistencia,10:00 ciclos,10:00 asistencia,11:00 ciclos,11:00 asistencia,12:00 ciclos,12:00 asistencia,13:00 ciclos,...,fecha,es_festivo,nombre_festivo,temperatura_max,temperatura_min,condiciones_cielo,prob_precipitacion,promedio_asistencia_por_hora,promedio_ciclos_por_hora,source_file
0,Balloon Wheel,0,0,6,61,6,98,6,102,6,...,2022-01-01,True,Año Nuevo,26.0,13.0,Patchy rain possible,0,140.000000,8.727273,01012022_limpio.csv
1,Samba Balloon,0,0,4,29,0,0,0,0,0,...,2022-01-01,True,Año Nuevo,26.0,13.0,Patchy rain possible,0,5.272727,0.727273,01012022_limpio.csv
2,Loco Bus,0,0,7,72,7,74,10,104,11,...,2022-01-01,True,Año Nuevo,26.0,13.0,Patchy rain possible,0,165.818182,14.363636,01012022_limpio.csv
3,Dragon,0,0,14,221,13,193,14,218,14,...,2022-01-01,True,Año Nuevo,26.0,13.0,Patchy rain possible,0,283.636364,18.545455,01012022_limpio.csv
4,Comanche,0,0,8,111,6,111,9,172,9,...,2022-01-01,True,Año Nuevo,26.0,13.0,Patchy rain possible,0,220.545455,12.363636,01012022_limpio.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15295,El Revoloteo,7,139,8,188,8,181,8,180,8,...,2024-10-31,False,Ninguno,23.0,15.0,Patchy rain possible,0,257.818182,11.454545,31102024_limpio.csv
15296,Guerra Pirata,6,65,11,145,9,182,12,244,11,...,2024-10-31,False,Ninguno,23.0,15.0,Patchy rain possible,0,238.727273,14.000000,31102024_limpio.csv
15297,Faro Saltarín,11,108,12,137,12,143,12,131,11,...,2024-10-31,False,Ninguno,23.0,15.0,Patchy rain possible,0,191.636364,16.909091,31102024_limpio.csv
15298,El Relámpago,22,184,30,262,30,268,30,278,30,...,2024-10-31,False,Ninguno,23.0,15.0,Patchy rain possible,0,356.545455,41.272727,31102024_limpio.csv


In [3]:

# put date in first column
all_data.insert(0, 'date', all_data['fecha'])
# remove 'fecha' column
all_data.drop(columns=['fecha'], inplace=True)

In [4]:
#convert 'date' to datetime
all_data['date'] = pd.to_datetime(all_data['date'], format='%Y-%m-%d')

all_data['day'] = all_data['date'].dt.day
all_data['month'] = all_data['date'].dt.month
all_data['year'] = all_data['date'].dt.year
all_data['day_of_week'] = all_data['date'].dt.day_name()

In [5]:
def es_temporada_alta(fecha):
    """
    Determina si una fecha está en temporada alta
    1 = Temporada alta, 0 = Temporada normal
    """
    mes = fecha.month
    dia = fecha.day
    
    # Temporada Navideña (Octubre, Noviembre, Diciembre)
    if mes in [11, 12]:
        return 1
    
    # Semana Santa ( marzo-abril)
    if mes == 3 and dia >= 15:  # Segunda quincena de marzo
        return 1
    if mes == 4 and dia <= 15:  # Primera quincena de abril
        return 1
    
    # Vacaciones de medio año
    if mes == 6 and dia >= 15:  # Segunda quincena de junio
        return 1
    
    return 0

all_data['temporada_alta'] = all_data['date'].apply(es_temporada_alta)

print("Distribución de temporada alta:")
print(all_data['temporada_alta'].value_counts())
print(f"\nPorcentaje temporada alta: {(all_data['temporada_alta'].sum() / len(all_data) * 100):.1f}%")

Distribución de temporada alta:
temporada_alta
0    10125
1     5175
Name: count, dtype: int64

Porcentaje temporada alta: 33.8%


In [6]:
all_data

,date,juego,9:00 ciclos,9:00 asistencia,10:00 ciclos,10:00 asistencia,11:00 ciclos,11:00 asistencia,12:00 ciclos,12:00 asistencia,...,condiciones_cielo,prob_precipitacion,promedio_asistencia_por_hora,promedio_ciclos_por_hora,source_file,day,month,year,day_of_week,temporada_alta
0,2022-01-01,Balloon Wheel,0,0,6,61,6,98,6,102,...,Patchy rain possible,0,140.000000,8.727273,01012022_limpio.csv,1,1,2022,Saturday,0
1,2022-01-01,Samba Balloon,0,0,4,29,0,0,0,0,...,Patchy rain possible,0,5.272727,0.727273,01012022_limpio.csv,1,1,2022,Saturday,0
2,2022-01-01,Loco Bus,0,0,7,72,7,74,10,104,...,Patchy rain possible,0,165.818182,14.363636,01012022_limpio.csv,1,1,2022,Saturday,0
3,2022-01-01,Dragon,0,0,14,221,13,193,14,218,...,Patchy rain possible,0,283.636364,18.545455,01012022_limpio.csv,1,1,2022,Saturday,0
4,2022-01-01,Comanche,0,0,8,111,6,111,9,172,...,Patchy rain possible,0,220.545455,12.363636,01012022_limpio.csv,1,1,2022,Saturday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15295,2024-10-31,El Revoloteo,7,139,8,188,8,181,8,180,...,Patchy rain possible,0,257.818182,11.454545,31102024_limpio.csv,31,10,2024,Thursday,0
15296,2024-10-31,Guerra Pirata,6,65,11,145,9,182,12,244,...,Patchy rain possible,0,238.727273,14.000000,31102024_limpio.csv,31,10,2024,Thursday,0
15297,2024-10-31,Faro Saltarín,11,108,12,137,12,143,12,131,...,Patchy rain possible,0,191.636364,16.909091,31102024_limpio.csv,31,10,2024,Thursday,0
15298,2024-10-31,El Relámpago,22,184,30,262,30,268,30,278,...,Patchy rain possible,0,356.545455,41.272727,31102024_limpio.csv,31,10,2024,Thursday,0


In [7]:
#export to csv
all_data.to_csv('all_data.csv', index=False)

In [8]:
# data types
all_data.dtypes

date                            datetime64[ns]
juego                                   object
9:00 ciclos                              int64
9:00 asistencia                          int64
10:00 ciclos                             int64
10:00 asistencia                         int64
11:00 ciclos                             int64
11:00 asistencia                         int64
12:00 ciclos                             int64
12:00 asistencia                         int64
13:00 ciclos                             int64
13:00 asistencia                         int64
14:00 ciclos                             int64
14:00 asistencia                         int64
15:00 ciclos                           float64
15:00 asistencia                       float64
16:00 ciclos                           float64
16:00 asistencia                       float64
17:00 ciclos                           float64
17:00 asistencia                       float64
18:00 ciclos                           float64
18:00 asisten

In [9]:
print(all_data['juego'].unique())

['Balloon Wheel' 'Samba Balloon' 'Loco Bus' 'Dragon' 'Comanche'
 'Ratón Loroco' 'Remolino' 'Rascacielos' 'Tronco Splash' 'Bumperazo'
 'Brincanguro' 'Moto Bala' 'Convoy' 'Bumpercitos' 'Bici Magica'
 'Polo Norte' 'Sol De Mi Barrio' 'Tifón' 'Carrusel' 'Trencito'
 'El Revoloteo' 'Guerra Pirata' 'Faro Saltarín' 'El Relámpago' 'Casichoco']


In [ ]:
yearly_attendance = all_data.groupby("year")["asistencia total"].sum()
print(yearly_attendance)

year
2022    3265587.0
2023    7536314.0
2024    7356626.0
Name: asistencia total, dtype: float64


In [ ]:
df_dia = all_data.groupby(["date", "day_of_week"])["asistencia total"].sum().reset_index()
print(df_dia)

average_by_day = df_dia.groupby("day_of_week")["asistencia total"].mean().sort_values(ascending=False)
print(average_by_day)

          date day_of_week  asistencia total
0   2022-01-01    Saturday           31071.0
1   2022-01-02      Sunday           34378.0
2   2022-01-06    Thursday           20391.0
3   2022-01-07      Friday           17696.0
4   2022-01-08    Saturday           24610.0
..         ...         ...               ...
605 2024-12-26    Thursday           33029.0
606 2024-12-27      Friday           36031.0
607 2024-12-28    Saturday           34942.0
608 2024-12-29      Sunday           41729.0
609 2024-12-30      Monday           28027.0

[610 rows x 3 columns]
day_of_week
Sunday       37335.708955
Saturday     33705.492537
Friday       27377.296296
Thursday     26787.323529
Monday       20206.166667
Tuesday      19704.777778
Wednesday    15874.517241
Name: asistencia total, dtype: float64


In [11]:
CAPACIDADES_JUEGOS = {
    "Balloon Wheel": 24,
    "Samba Ballon": 32,
    "Loco Bus": 24,
    "Dragón": 20,
    "Comanche": 24,
    "Ratón Loroco": 4,
    "Remolino": 6,
    "Rascacielos": 18,
    "Tronco Splash": 4,
    "Bumperazo": 60,
    "Brinkanguro": 24,
    "Moto Bala": 12,
    "Convoy": 20,
    "Bumpercitos": 28,
    "Bici Mágica": 12,
    "Polo Norte": 12,
    "Sol de Mi Barrio": 108,
    "Tifón": 16,
    "Carrusel": 52,
    "Trencito": 74,
    "Revoloteo": 24,
    "Guerra Pirata": 24,
    "Faro Saltarín": 16,
    "Relámpago": 9,
    "Casichoco": 40
}


In [12]:
MAX_TIME_JUEGOS = {
    "Balloon Wheel": 60,
    "Samba Ballon": 60,
    "Loco Bus": 60,
    "Dragón": 60,
    "Comanche": 90,
    "Ratón Loroco": 90,
    "Remolino": 90,
    "Rascacielos": 90,
    "Tronco Splash": 90,
    "Bumperazo": 60,
    "Brinkanguro": 60,
    "Moto Bala": 90,
    "Convoy": 60,
    "Bumpercitos": 60,
    "Bici Mágica": 60,
    "Polo Norte": 60,
    "Sol de Mi Barrio":  90,
    "Tifón": 90,
    "Carrusel": 60,
    "Trencito": 90,
    "Revoloteo": 60,
    "Guerra Pirata": 60,
    "Faro Saltarín": 60,
    "Relámpago" :90,
    "Casichoco": 60

}

In [13]:
BETA_JUEGO = {
    "Balloon Wheel": 0.73,
    "Bumperazo": 0.44,
    "Bumpercitos": 0.49,
    "Brincanguro": 0.98,
    "Bici Magica": 0.94,
    "Carrusel": 0.53,
    "Casichoco": 0.61,
    "Comanche": 0.98,
    "Convoy": 0.51,
    "Dragón": 0.995,
    "Relámpago": 0.992,
    "Faro Saltarín": 0.806,
    "Guerra Pirata": 0.901,
    "Loco Bus": 0.716,
    "Moto Bala": 1.00,
    "Polo Norte": 0.6,
    "Rascacielos": 1.00,
    "Ratón Loroco": 0.937,
    "Remolino": 0.944,
    "Samba Balloon": 0.511,
    "Tifón": 0.95,
    "Trencito": 0.536,
    "Tronco Splash": 0.764,
}

In [14]:
def get_beta(juego_name: str) -> float:
    return float(BETA_JUEGO.get(juego_name, 0.80))

### DF by game

In [15]:
"""
['Balloon Wheel' 'Samba Balloon' 'Loco Bus' 'Dragon' 'Comanche'
 'Ratón Loroco' 'Remolino' 'Rascacielos' 'Tronco Splash' 'Bumperazo'
 'Brincanguro' 'Moto Bala' 'Convoy' 'Bumpercitos' 'Bici Magica'
 'Polo Norte' 'Sol De Mi Barrio' 'Tifón' 'Carrusel' 'Trencito'
 'El Revoloteo' 'Guerra Pirata' 'Faro Saltarín' 'El Relámpago' 'Casichoco']
"""

df_balloon_wheel = all_data[all_data['juego'] == 'Balloon Wheel']
df_balloon_wheel = df_balloon_wheel.drop(columns=['juego'])

df_samba_ballon = all_data[all_data['juego'] == 'Samba Balloon']
df_samba_ballon = df_samba_ballon.drop(columns=['juego'])

df_loco_bus = all_data[all_data['juego'] == 'Loco Bus']
df_loco_bus = df_loco_bus.drop(columns=['juego'])

df_dragon = all_data[all_data['juego'] == 'Dragon']
df_dragon = df_dragon.drop(columns=['juego'])

df_comanche = all_data[all_data['juego'] == 'Comanche']
df_comanche = df_comanche.drop(columns=['juego'])    

df_raton_loroco = all_data[all_data['juego'] == 'Ratón Loroco']
df_raton_loroco = df_raton_loroco.drop(columns=['juego'])

df_remolino = all_data[all_data['juego'] == 'Remolino']
df_remolino = df_remolino.drop(columns=['juego'])

df_rascacielos = all_data[all_data['juego'] == 'Rascacielos']
df_rascacielos = df_rascacielos.drop(columns=['juego'])

df_tronco_splash = all_data[all_data['juego'] == 'Tronco Splash']
df_tronco_splash = df_tronco_splash.drop(columns=['juego'])

df_bumperazo = all_data[all_data['juego'] == 'Bumperazo']
df_bumperazo = df_bumperazo.drop(columns=['juego'])

df_brincanguro = all_data[all_data['juego'] == 'Brincanguro']
df_brincanguro = df_brincanguro.drop(columns=['juego'])

df_moto_bala = all_data[all_data['juego'] == 'Moto Bala']
df_moto_bala = df_moto_bala.drop(columns=['juego'])

df_convoy = all_data[all_data['juego'] == 'Convoy']
df_convoy = df_convoy.drop(columns=['juego'])

df_bumpercitos = all_data[all_data['juego'] == 'Bumpercitos']
df_bumpercitos = df_bumpercitos.drop(columns=['juego'])

df_bici_magica = all_data[all_data['juego'] == 'Bici Magica']
df_bici_magica = df_bici_magica.drop(columns=['juego'])

df_polo_norte = all_data[all_data['juego'] == 'Polo Norte']
df_polo_norte = df_polo_norte.drop(columns=['juego'])

df_sol_de_mi_barrio = all_data[all_data['juego'] == 'Sol De Mi Barrio']
df_sol_de_mi_barrio = df_sol_de_mi_barrio.drop(columns=['juego'])

df_tifon = all_data[all_data['juego'] == 'Tifón']
df_tifon = df_tifon.drop(columns=['juego'])

df_carrusel = all_data[all_data['juego'] == 'Carrusel']
df_carrusel = df_carrusel.drop(columns=['juego'])

df_trencito = all_data[all_data['juego'] == 'Trencito']
df_trencito = df_trencito.drop(columns=['juego'])

df_el_revoloteo = all_data[all_data['juego'] == 'El Revoloteo']
df_el_revoloteo = df_el_revoloteo.drop(columns=['juego'])

df_guerra_pirata = all_data[all_data['juego'] == 'Guerra Pirata']
df_guerra_pirata = df_guerra_pirata.drop(columns=['juego'])

df_faro_saltarin = all_data[all_data['juego'] == 'Faro Saltarín']
df_faro_saltarin = df_faro_saltarin.drop(columns=['juego'])

df_el_relampago = all_data[all_data['juego'] == 'El Relámpago']
df_el_relampago = df_el_relampago.drop(columns=['juego'])

df_casichoco = all_data[all_data['juego'] == 'Casichoco']
df_casichoco = df_casichoco.drop(columns=['juego'])


In [16]:
df_balloon_wheel

,date,9:00 ciclos,9:00 asistencia,10:00 ciclos,10:00 asistencia,11:00 ciclos,11:00 asistencia,12:00 ciclos,12:00 asistencia,13:00 ciclos,...,condiciones_cielo,prob_precipitacion,promedio_asistencia_por_hora,promedio_ciclos_por_hora,source_file,day,month,year,day_of_week,temporada_alta
0,2022-01-01,0,0,6,61,6,98,6,102,6,...,Patchy rain possible,0,140.000000,8.727273,01012022_limpio.csv,1,1,2022,Saturday,0
25,2022-04-01,0,0,11,32,7,37,8,47,7,...,Overcast,0,49.636364,9.818182,01042022_limpio.csv,1,4,2022,Friday,1
50,2022-05-01,6,72,6,105,6,107,6,108,6,...,Patchy rain possible,0,164.545455,9.818182,01052022_limpio.csv,1,5,2022,Sunday,0
75,2022-07-01,0,0,0,0,0,0,0,0,6,...,Patchy rain possible,0,59.636364,3.636364,01072022_limpio.csv,1,7,2022,Friday,0
100,2022-09-01,1,10,4,13,6,52,7,53,7,...,Moderate or heavy rain shower,0,48.727273,7.818182,01092022_limpio.csv,1,9,2022,Thursday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15175,2024-12-30,6,28,6,63,6,87,6,81,6,...,Moderate rain at times,0,96.363636,8.727273,30122024_limpio.csv,30,12,2024,Monday,1
15200,2024-03-31,4,26,6,82,6,99,6,106,6,...,Overcast,0,143.818182,9.454545,31032024_limpio.csv,31,3,2024,Sunday,1
15225,2024-05-31,4,69,6,94,6,114,6,119,6,...,Patchy rain possible,0,147.454545,8.363636,31052024_limpio.csv,31,5,2024,Friday,0
15250,2024-08-31,6,38,6,89,6,105,3,53,0,...,Patchy light rain with thunder,0,63.090909,4.545455,31082024_limpio.csv,31,8,2024,Saturday,0


## M/M/1

In [17]:
import numpy as np
import pandas as pd

HORAS = [f"{h}:00" for h in range(9, 19)]  # 9..18

def mm1_perhour(df, capacidad_por_ciclo, max_espera_min=60, juego_name=None):
    df = df.copy()
    beta = get_beta(juego_name) if juego_name is not None else 0.80

    for h in HORAS:
        col_cic = f"{h} ciclos"
        col_asist = f"{h} asistencia"
        if col_cic not in df.columns or col_asist not in df.columns:
            continue

        lam = df[col_asist].astype(float).to_numpy() / 60.0
        mu  = (df[col_cic].astype(float).to_numpy() * (float(capacidad_por_ciclo) * beta)) / 60.0

        rho   = np.full(len(df), np.nan)
        Wq    = np.full(len(df), np.nan)
        state = np.array([""] * len(df), dtype=object)

        closed       = (mu == 0) & (lam == 0)
        off_service  = (mu == 0) & (lam > 0)
        stable       = (mu > lam) & (mu > 0)
        saturated    = (mu <= lam) & (mu > 0)

        rho[closed] = 0
        Wq[closed]  = 0
        state[closed] = "Closed"

        rho[off_service] = np.inf
        Wq[off_service]  = max_espera_min
        state[off_service] = "Off service"

        rho[stable] = lam[stable] / mu[stable]
        Wq[stable]  = lam[stable] / (mu[stable] * (mu[stable] - lam[stable]))
        state[stable] = "stable"

        rho[saturated] = lam[saturated] / mu[saturated]
        Wq[saturated]  = max_espera_min
        state[saturated] = "saturated"

        df[f"lambda_{h}(/min)"] = lam.round(4)
        df[f"mu_{h}(/min)"]     = mu.round(4)
        df[f"rho_{h}"]          = np.clip(rho, None, 10)
        df[f"Wq_{h}(min)"]      = np.minimum(Wq, max_espera_min)
        df[f"state_{h}"]        = state
    return df


In [18]:
ballon_wheel_finaldf = mm1_perhour(df_balloon_wheel, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Balloon Wheel"], max_espera_min=MAX_TIME_JUEGOS["Balloon Wheel"], juego_name="Balloon Wheel")
samba_ballon_finaldf = mm1_perhour(df_samba_ballon, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Samba Ballon"], max_espera_min=MAX_TIME_JUEGOS["Samba Ballon"], juego_name="Samba Ballon")
loco_bus_finaldf     = mm1_perhour(df_loco_bus,     capacidad_por_ciclo=CAPACIDADES_JUEGOS["Loco Bus"],       max_espera_min=MAX_TIME_JUEGOS["Loco Bus"],       juego_name="Loco Bus")
dragon_finaldf       = mm1_perhour(df_dragon,       capacidad_por_ciclo=CAPACIDADES_JUEGOS["Dragón"],         max_espera_min=MAX_TIME_JUEGOS["Dragón"],         juego_name="Dragón")
comanche_finaldf     = mm1_perhour(df_comanche,     capacidad_por_ciclo=CAPACIDADES_JUEGOS["Comanche"],       max_espera_min=MAX_TIME_JUEGOS["Comanche"],       juego_name="Comanche")
raton_loroco_finaldf = mm1_perhour(df_raton_loroco, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Ratón Loroco"],   max_espera_min=MAX_TIME_JUEGOS["Ratón Loroco"],   juego_name="Ratón Loroco")
remolino_finaldf     = mm1_perhour(df_remolino,     capacidad_por_ciclo=CAPACIDADES_JUEGOS["Remolino"],       max_espera_min=MAX_TIME_JUEGOS["Remolino"],       juego_name="Remolino")
rascacielos_finaldf  = mm1_perhour(df_rascacielos,  capacidad_por_ciclo=CAPACIDADES_JUEGOS["Rascacielos"],    max_espera_min=MAX_TIME_JUEGOS["Rascacielos"],    juego_name="Rascacielos")
tronco_splash_finaldf= mm1_perhour(df_tronco_splash,capacidad_por_ciclo=CAPACIDADES_JUEGOS["Tronco Splash"],  max_espera_min=MAX_TIME_JUEGOS["Tronco Splash"],  juego_name="Tronco Splash")
bumperazo_finaldf    = mm1_perhour(df_bumperazo,    capacidad_por_ciclo=CAPACIDADES_JUEGOS["Bumperazo"],      max_espera_min=MAX_TIME_JUEGOS["Bumperazo"],      juego_name="Bumperazo")
brincanguro_finaldf  = mm1_perhour(df_brincanguro,  capacidad_por_ciclo=CAPACIDADES_JUEGOS["Brinkanguro"],    max_espera_min=MAX_TIME_JUEGOS["Brinkanguro"],    juego_name="Brinkanguro")
moto_bala_finaldf    = mm1_perhour(df_moto_bala,    capacidad_por_ciclo=CAPACIDADES_JUEGOS["Moto Bala"],      max_espera_min=MAX_TIME_JUEGOS["Moto Bala"],      juego_name="Moto Bala")
convoy_finaldf       = mm1_perhour(df_convoy,       capacidad_por_ciclo=CAPACIDADES_JUEGOS["Convoy"],         max_espera_min=MAX_TIME_JUEGOS["Convoy"],         juego_name="Convoy")
bumpercitos_finaldf  = mm1_perhour(df_bumpercitos,  capacidad_por_ciclo=CAPACIDADES_JUEGOS["Bumpercitos"],    max_espera_min=MAX_TIME_JUEGOS["Bumpercitos"],    juego_name="Bumpercitos")
bici_magica_finaldf  = mm1_perhour(df_bici_magica,  capacidad_por_ciclo=CAPACIDADES_JUEGOS["Bici Mágica"],    max_espera_min=MAX_TIME_JUEGOS["Bici Mágica"],    juego_name="Bici Mágica")
polo_norte_finaldf   = mm1_perhour(df_polo_norte,   capacidad_por_ciclo=CAPACIDADES_JUEGOS["Polo Norte"],     max_espera_min=MAX_TIME_JUEGOS["Polo Norte"],     juego_name="Polo Norte")
sol_de_mi_barrio_finaldf = mm1_perhour(df_sol_de_mi_barrio, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Sol de Mi Barrio"], max_espera_min=MAX_TIME_JUEGOS["Sol de Mi Barrio"], juego_name="Sol de Mi Barrio")
tifon_finaldf        = mm1_perhour(df_tifon,        capacidad_por_ciclo=CAPACIDADES_JUEGOS["Tifón"],          max_espera_min=MAX_TIME_JUEGOS["Tifón"],          juego_name="Tifón")
carrusel_finaldf     = mm1_perhour(df_carrusel,     capacidad_por_ciclo=CAPACIDADES_JUEGOS["Carrusel"],       max_espera_min=MAX_TIME_JUEGOS["Carrusel"],       juego_name="Carrusel")
trencito_finaldf     = mm1_perhour(df_trencito,     capacidad_por_ciclo=CAPACIDADES_JUEGOS["Trencito"],       max_espera_min=MAX_TIME_JUEGOS["Trencito"],       juego_name="Trencito")
el_revoloteo_finaldf = mm1_perhour(df_el_revoloteo, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Revoloteo"],      max_espera_min=MAX_TIME_JUEGOS["Revoloteo"],      juego_name="Revoloteo")
guerra_pirata_finaldf= mm1_perhour(df_guerra_pirata,capacidad_por_ciclo=CAPACIDADES_JUEGOS["Guerra Pirata"],  max_espera_min=MAX_TIME_JUEGOS["Guerra Pirata"],  juego_name="Guerra Pirata")
faro_saltarin_finaldf= mm1_perhour(df_faro_saltarin,capacidad_por_ciclo=CAPACIDADES_JUEGOS["Faro Saltarín"],  max_espera_min=MAX_TIME_JUEGOS["Faro Saltarín"],  juego_name="Faro Saltarín")
el_relampago_finaldf = mm1_perhour(df_el_relampago, capacidad_por_ciclo=CAPACIDADES_JUEGOS["Relámpago"],      max_espera_min=MAX_TIME_JUEGOS["Relámpago"],      juego_name="Relámpago")
casichoco_finaldf    = mm1_perhour(df_casichoco,    capacidad_por_ciclo=CAPACIDADES_JUEGOS["Casichoco"],      max_espera_min=MAX_TIME_JUEGOS["Casichoco"],      juego_name="Casichoco")

In [19]:
ballon_wheel_finaldf.to_csv("./data/by_game/ballon_wheel.csv" , index=False)
samba_ballon_finaldf.to_csv("./data/by_game/samba_ballon.csv" , index=False)
loco_bus_finaldf.to_csv("./data/by_game/loco_bus.csv" , index=False)
dragon_finaldf.to_csv("./data/by_game/dragon.csv" , index=False)
comanche_finaldf.to_csv("./data/by_game/comanche.csv" , index=False)
raton_loroco_finaldf.to_csv("./data/by_game/raton_loroco.csv" , index=False)
remolino_finaldf.to_csv("./data/by_game/remolino.csv" , index=False)
rascacielos_finaldf.to_csv("./data/by_game/rascacielos.csv" , index=False)
tronco_splash_finaldf.to_csv("./data/by_game/tronco_splash.csv" , index=False)
bumperazo_finaldf.to_csv("./data/by_game/bumperazo.csv" , index=False)
brincanguro_finaldf.to_csv("./data/by_game/brincanguro.csv" , index=False)
moto_bala_finaldf.to_csv("./data/by_game/moto_bala.csv" , index=False)
convoy_finaldf.to_csv("./data/by_game/convoy.csv" , index=False)
bumpercitos_finaldf.to_csv("./data/by_game/bumpercitos.csv" , index=False)
bici_magica_finaldf.to_csv("./data/by_game/bici_magica.csv" , index=False)
polo_norte_finaldf.to_csv("./data/by_game/polo_norte.csv" , index=False)
sol_de_mi_barrio_finaldf.to_csv("./data/by_game/sol_de_mi_barrio.csv" , index=False)
tifon_finaldf.to_csv("./data/by_game/tifon.csv" , index=False)
carrusel_finaldf.to_csv("./data/by_game/carrusel.csv" , index=False)
trencito_finaldf.to_csv("./data/by_game/trencito.csv" , index=False)
el_revoloteo_finaldf.to_csv("./data/by_game/el_revoloteo.csv" , index=False)
guerra_pirata_finaldf.to_csv("./data/by_game/guerra_pirata.csv" , index=False)
faro_saltarin_finaldf.to_csv("./data/by_game/faro_saltarin.csv" , index=False)
el_relampago_finaldf.to_csv("./data/by_game/el_relampago.csv" , index=False)
casichoco_finaldf.to_csv("./data/by_game/casichoco.csv" , index=False)